In [ ]:
from ipywidgets import Image

from ipywidgets import ColorPicker,Button, IntSlider, link, AppLayout, HBox

from ipycanvas import Canvas, hold_canvas

import time


<h1>OCR PROJECTSSSS</h1>

In [ ]:
width = 1000
height = 1000

In [ ]:
import numpy as np

number_comparisons = int(input("How many comparisons would you like to execute for your image (60000 will take around 10 seconds, 10000 would take around 3 seconds). Accuracy significantly decreases with less comparisons"))
width = 784
height = 784
cell_size = 28

canvas = Canvas(width=width, height=height,sync_image_data=True)
canvas.stroke_rect(0, 0, width, height)
canvas.sync_image_data = True
drawing = False
position = None


TEST_DATA_FILENAME = "t10k-images.idx3-ubyte"
TEST_LABELS_FILENAME = "t10k-labels.idx1-ubyte"
TRAIN_DATA_FILENAME = "train-images.idx3-ubyte"
TRAIN_LABELS_FILENAME = "train-labels.idx1-ubyte"
DEBUG = True

def read_labels(filename,n_max_labels = None):
    labels = [] #variable que guarda todas las imagenes
    with open(filename, "rb") as f: #abrir el fichero filename como f, y leerlo en binario ("rb")
        _  =f.read(4) #numero inutil (representa algo que no necesitamos)
        #los siguientes 12 bytes representan el numero de imagenes, el numero de filas y de columnas
        n_labels = bytes_to_int(f.read(4))
        if n_max_labels:
            n_labels = n_max_labels
        for label_idx in range(n_labels):
            label = f.read(1)
            labels.append(label)
            
    return labels

count = 0

def read_images(filename,n_max_images = None):
    global count
    images = [] #variable que guarda todas las imagenes
    with open(filename, "rb") as f: #abrir el fichero filename como f, y leerlo en binario ("rb")
        _  =f.read(4) #numero inutil (representa algo que no necesitamos)
        #los siguientes 12 bytes representan el numero de imagenes, el numero de filas y de columnas
        n_images = bytes_to_int(f.read(4))
        if n_max_images:
            n_images = n_max_images
        n_rows = bytes_to_int(f.read(4))
        n_columns = bytes_to_int(f.read(4))
        for image_idx in range(n_images):
            image = []#variable que guarda la imagen actual
            for row_idx in range(n_rows):
                row = []#variable que guarda la columna actual
                for column_idx in range(n_columns):
                    count += 1
                    pixel = f.read(1) #leemos el pixel actual de 8 bits y lo apendizamos a la row
                    row.append(pixel)
                image.append(row)#metemos la row en la image
            images.append(image)#metemos la image en el conjunto de images
    return images

def aplanar_lista(l):
    return [pixel for sublist in l for pixel in sublist]
def bytes_to_int(byte_data):
    if byte_data == 0:
        return 0
    elif byte_data == 255:
        return 255
    else:
        return int.from_bytes(byte_data,"big")

def pasar_lista_unidimensional(X):
    return [aplanar_lista(sample) for sample in X]

def dist(x,y):
    return sum((bytes_to_int(x_i) - bytes_to_int(y_i)) **2 for x_i,y_i in zip(x,y))**0.5 #distancia euclides
def distancia_entre_samples(X_train,test_sample):
    return [dist(train_sample,test_sample) for train_sample in X_train] #por todas las imagenes, calculamos su distancia arriba

def most_frequent_element(list):
    return max(list, key= list.count)


def knn(X_train,y_train,X_test, k = 3):
    y_pred = [] #la prediccion que tenemos a los x_test
    for test_sample_idx,test_sample in enumerate(X_test):
        training_distances = distancia_entre_samples(X_train,test_sample) #queremos conseguir las distancias a todos los puntos
        sorted_distance_indices = [
            pair[0]
            for pair in sorted(enumerate(training_distances), key = lambda x: x[1]) ]#escogemos la menor distancia
        candidates = [bytes_to_int(y_train[idx]) for idx in sorted_distance_indices[:k]] # k mejores candidatos
        print("Top k choices were", candidates)
        top_candidate = most_frequent_element(candidates)
        y_pred.append(top_candidate) #apuntamos a predicción
    return y_pred
def main():
    global X_test

    #"X" es igual al dataset y "y" es el label asignado
    X_train = read_images(TRAIN_DATA_FILENAME,number_comparisons)
    y_train = read_labels(TRAIN_LABELS_FILENAME,number_comparisons)
    y_test = read_labels(TEST_LABELS_FILENAME,1)

    

    X_train = pasar_lista_unidimensional(X_train) #queremos pasar la matriz de valores a una matriz unidimensional
    k1 = int(input("How many k nearest neighbors would you like(3 is recommended, anything above will change accuracy): "))
    y_pred = knn(X_train,y_train,X_test,k)
    print("The number you have just written is: " ,y_pred)
    print("Number of iterations: ", count)
    return
 

#here starts the DRAWING CODE
#-----------------------------------------------------------------------
def get_cell_coordinates(x, y):
    cell_x = int(x / cell_size) * cell_size
    cell_y = int(y / cell_size) * cell_size
    return cell_x, cell_y


def on_mouse_down(x, y):
    global drawing
    global position

    drawing = True
    position = get_cell_coordinates(x, y)


def on_mouse_move(x, y):
    global drawing
    global position
    if not drawing:
        return
    
    cell_x, cell_y = get_cell_coordinates(x, y)
    canvas.fill_rect(cell_x-28, cell_y, cell_size*3, cell_size)
    canvas.fill_rect(cell_x, cell_y-28, cell_size, cell_size*3)

    position = (cell_x, cell_y)


def on_mouse_up(x, y):
    global drawing
    global position

    drawing = False

def process_canvas(button):
    global processed_array,X_test
    # Obtener el contenido del canvas como un array de 28x28
    canvas_data = canvas.get_image_data(0, 0, width, height)
    canvas_array = np.frombuffer(canvas_data.data, dtype=np.uint8).reshape((height, width, 4))
    # Convertir a escala de grises
    gray_array = canvas_array[:, :, 3]
    # Crear un array de 28x28 con 0 donde hay dibujo y 255 donde no hay dibujo
    processed_array = [[]]
    for i in range(28):
        for j in range(28):
            processed_array[0].append(255 if gray_array[i * (height // 28), j * (width // 28)] == 255 else 0)
    X_test = processed_array
    start_time = time.time()
    main()
    end_time = time.time()
    print("Time that took to compare:" ,round(end_time-start_time,5)," Seconds")
    return X_test

    # Imprimir el resultado



canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)
picker = ColorPicker(description="Color:", value="#000000")

link((picker, "value"), (canvas, "stroke_style"))
link((picker, "value"), (canvas, "fill_style"))

process_button = Button(description="Procesar")
X_test = process_button.on_click(process_canvas)
HBox((canvas, picker, process_button))
